Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [1]:
from azureml.core import Workspace
ws = Workspace.get(name=<<your workspace name>>, subscription_id='4feb84f6-2c10-4536-9c8a-0a2360eabfc5', resource_group='azureml')

## Register a small magic to more conveniently write out a file

In [2]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Get the datastore to mount and set the target and temp paths

In [3]:
ds = ws.get_default_datastore()
mount_path='~/data/amldefaultdatastore'
temp_path='~/data/temp'

## Write out scripts that do the mount and umount for the datastore

In [4]:
%%writetemplate mount_{ds.name}.sh
mkdir -p {mount_path}
mkdir -p {temp_path}
export AZURE_STORAGE_ACCOUNT={ds.account_name}
export AZURE_STORAGE_ACCESS_KEY={ds.account_key}
blobfuse {mount_path} --container-name={ds.container_name} --tmp-path={temp_path}

In [5]:
%%writetemplate umount_{ds.name}.sh
fusermount -u {mount_path}

## Try out the scripts

In [13]:
!sh mount_{ds.name}.sh

In [14]:
!ls {mount_path} -alg

total 4
drwxrwx--- 2 danielsc 4096 Nov  1 21:27 .
drwxrwxr-x 4 danielsc 4096 Nov  1 04:50 ..
drwxrwx--- 3 danielsc 4096 Jan  1  1970 imagenet_samples
drwxrwx--- 3 danielsc 4096 Jan  1  1970 mnist
drwxrwx--- 3 danielsc 4096 Jan  1  1970 mnistx


In [12]:
!sh umount_{ds.name}.sh

In [15]:
ls 

~/       mount-datastore.ipynb        README.md
LICENSE  mount_workspaceblobstore.sh  umount_workspaceblobstore.sh
